In [0]:
df = spark.read.format("delta") \
    . option("header", "true") \
    . option("inferSchema", "true") \
    . load("abfss://bronze-layer@netflixprojectcharan.dfs.core.windows.net/netflix_titles")

In [0]:
df.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# from pyspark.sql.functions import avg
# df_duratio_min_mean = df.agg(avg('duration_minutes')).collect()[0][0]


In [0]:
df=df.fillna({"duration_minutes":0,"duration_seasons":1})

In [0]:
df.display()

In [0]:
df = df.withColumn("duration_minutes", col("duration_minutes").cast("int")) \
    .withColumn("duration_seasons", col("duration_seasons").cast("int"))
# df = df.withColumn("duration_seasons", col("duration_seasons").cast("int"))")

In [0]:
df.display()
# df.write.mode("overwrite")

In [0]:
df = df.withColumn("short_title",split(col('title'),':')[0])\
    .withColumn('end_title',split(col('title'),':')[1])

In [0]:
df.display()

In [0]:
df = df.withColumn("rating",split(col('rating'),'-')[0])
df.display()

In [0]:
df = df.withColumn("type_flag",when(col('type')=="Movie",1).otherwise(0))
df.display()

In [0]:
from pyspark.sql.window import Window

df = df.withColumn('duration_ranking',dense_rank().over(Window.partitionBy().orderBy(col('duration_minutes').desc())))
df.display()

In [0]:
df.createOrReplaceTempView("temp_view")

In [0]:
df.createOrReplaceGlobalTempView("global_temp_view")

In [0]:
df_sql = spark.sql(""" 
                  
                  SELECT
                   * ,
                   dense_rank() over(ORDER BY duration_minutes DESC) AS dense_rank_through_sql
                  FROM temp_view
                --   WHERE rating = "TV"
                  
                  """)


In [0]:
df_sql_global = spark.sql(""" 
                  
                  SELECT
                   * ,
                   dense_rank() over(ORDER BY duration_minutes DESC) AS dense_rank_through_sql
                  FROM global_temp.global_temp_view
                --   WHERE rating = "TV"
                  
                  """)

In [0]:
df_sql.display()

In [0]:
df_sql_global.display()

In [0]:
df_viz = df.groupBy("type").agg(count('*')).alias('total_count')
df_viz.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df.display()

In [0]:
df.write.format("delta") \
    .mode("overwrite") \
    .option("path",'abfss://silver-layer@netflixprojectcharan.dfs.core.windows.net/netflix_titles') \
    .save()